# 0. Install

In [ ]:
pip install opencv-python

In [ ]:
pip install mediapipe

# 1. Library

In [1]:
import cv2
import mediapipe as mp
import csv
import os
import time

# 2. Mediapip Open(for GSN)

In [28]:
# 데이터 저장 폴더 설정
output_dir = r"data\gsn"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 파일명 설정
csv_filename_base = "469"  # 파일명 수정할 것
csv_filename = os.path.join(output_dir, f"{csv_filename_base}.csv")
video_filename = os.path.join(output_dir, f"{csv_filename_base}.avi")  

# 미디어파이프 설정
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

mp_drawing = mp.solutions.drawing_utils

# 웹캠 오픈
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# VideoWriter 설정
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(video_filename, fourcc, 20.0, (640, 480))

exit_requested = False
print("Continuous data collection started.")

with open(csv_filename, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)

    header = ["timestamp", "hand_detected"]
    for hand in ['hand1', 'hand2']:
        for i in range(21):
            header.extend([f"{hand}_landmark{i}_x", f"{hand}_landmark{i}_y", f"{hand}_landmark{i}_z"])
    writer.writerow(header)

    while not exit_requested:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not retrieve frame.")
            break

        frame = cv2.flip(frame, 1)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False
        results = hands.process(image_rgb)
        image_rgb.flags.writeable = True

        ts = time.time()
        row = [ts]

        if results.multi_hand_landmarks:
            hand_detected = 1
        else:
            hand_detected = 0
        row.append(hand_detected)

        if results.multi_hand_landmarks:
            landmarks_all = []
            for hand_landmarks in results.multi_hand_landmarks:
                landmarks = []
                for lm in hand_landmarks.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])
                landmarks_all.append(landmarks)
            row.extend(landmarks_all[0])
            if len(landmarks_all) >= 2:
                row.extend(landmarks_all[1])
            else:
                row.extend([0] * 63)
        else:
            row.extend([0] * 126)

        writer.writerow(row)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        cv2.imshow("Continuous Data Collection", frame)
        
        out.write(frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            exit_requested = True

# 끝
cap.release()
out.release()
cv2.destroyAllWindows()
hands.close()
print(f"Exiting. Data saved to {csv_filename} and {video_filename}")

Continuous data collection started.
Exiting. Data saved to data\gsn_new_2\469.csv
